In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (10, 9)
plt.style.use('seaborn-darkgrid')
sns.set_style('darkgrid')

from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression, Lasso, Ridge

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

from warnings import filterwarnings, simplefilter
filterwarnings('ignore')
simplefilter('ignore')

from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback

import gc
gc.enable()

In [ ]:
try :
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver().connect()
    strategy = tf.distribute.TPUStrategy(tpu)
except :
    strategy = tf.distribute.get_strategy()

In [ ]:
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')

In [ ]:
def make_dataset(x, y = None, batch_size = 512, shuffle = False) :
    def preprocess(x, y) :
        return x, y
    feat = [col for col in x.columns if col.startswith('f_')]
    feature = x[feat].values
    dataset = tf.data.Dataset.from_tensor_slices((
        feature, y
    )).map(preprocess).batch(batch_size = batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    if shuffle :
        dataset = dataset.shuffle(500, seed = 50)
    return dataset

In [ ]:
def correlationMetric(x, y, axis=-2):
    from tensorflow.python.ops import math_ops
    """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    return corr


def correlationLoss(x,y, axis=-2):
    from tensorflow.python.ops import math_ops
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xsqsum = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    ysqsum = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xsqsum * ysqsum)
    return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr ) , dtype=tf.float32 )

def build_model() :
    with strategy.scope() :
        inputs = keras.layers.Input(shape = (300, ), dtype = tf.float16)
        x = keras.layers.Dense(256, activation = 'swish')(inputs)
        x = keras.layers.Dropout(.1)(x)

        x = keras.layers.Reshape((-1, 1))(x)
        x = keras.layers.Conv1D(16, 4, strides = 1, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Conv1D(32, 4, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Conv1D(64, 4, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Conv1D(128, 4, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Conv1D(128, 4, padding = 'same')(x)
        x = keras.layers.MaxPool1D()(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Flatten()(x)

        x1 = keras.layers.Dense(256, activation = 'swish', kernel_regularizer = 'l2')(x)
        x2 = keras.layers.Dense(128, activation = 'swish', kernel_regularizer=  'l2')(x1)

        x = keras.layers.Concatenate(axis = -1)([x1, x2])
        x = keras.layers.Dense(64, activation = 'swish', kernel_regularizer = 'l2')(x)
        out = keras.layers.Dense(1, activation = 'linear')(x)

        model = keras.models.Model(
            inputs = inputs,
            outputs = out
        )
        model.compile(
            optimizer = keras.optimizers.Adam(learning_rate = 7e-4),
            loss = correlationLoss,
            metrics = [correlationMetric, keras.metrics.RootMeanSquaredError(name = 'rmse')]
        )
    return model

In [ ]:
model = build_model()
model.summary()
keras.utils.plot_model(model, show_shapes = True)

In [ ]:
cv_index = [
    (
        train.loc[(train.time_id > 800) & (train.time_id <= 1000)].index.values,
        train.loc[(train.time_id > 1000)].index.values
    ),
    (
        train.loc[(train.time_id > 900) & (train.time_id <= 1100)].index.values,
        train.loc[(train.time_id > 1100)].index.values
    ),
    (
        train.loc[(train.time_id > 1000) & (train.time_id <= 1200)].index.values,
        train.loc[(train.time_id > 1200)].index.values
    ),
    (
        train.loc[(train.time_id > 800) & (train.time_id <= 1200)].index.values,
        train.loc[(train.time_id > 1200)].index.values
    )
]

In [ ]:
y = train.pop('target')
y

In [ ]:
model_dir = "./mnist_model"

localhost_save_option = tf.saved_model.SaveOptions(experimental_io_device="/job:localhost")

In [ ]:
for i, (t, v) in enumerate(cv_index) :
    xtrain = train.iloc[t, :]
    xval = train.iloc[v, :]
    ytrain = y.iloc[t]
    yval = y.iloc[v]
    
    gc.collect()
    K.clear_session()
    model = build_model()
    cb = [
        keras.callbacks.EarlyStopping(patience = 14, restore_best_weights = True),
        keras.callbacks.ReduceLROnPlateau(patience = 4, factor = .3, min_lr = 1e-5),
        TqdmCallback(verbose = 1)
    ]
    train_ds = make_dataset(xtrain, ytrain, batch_size = 512, shuffle = True)
    val_ds = make_dataset(xval, yval, batch_size = 512, shuffle = False)
    del xtrain, ytrain
    history = model.fit(
        train_ds, validation_data = val_ds,
        callbacks = cb, epochs = 250, verbose = 0
    )
    history = pd.DataFrame(history.history).loc[2:, ['val_loss', 'loss']].plot.line(figsize = (10, 9))
    plt.show()
    yhat = model.predict(val_ds).ravel()
    score, p = pearsonr(yval, yhat)
    model.save_weights(f'cnn_model_fold_{i}', options=localhost_save_option)
    print(f'Pearson : {score}')
    print(f'p-value : {p}')
    del xval, yval, history, model
    gc.collect()